In [1]:
from pylab import axes
from matplotlib import ticker
import numpy as np
import pickle
import matplotlib
import pickle
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm, rc
from matplotlib.colors import LinearSegmentedColormap
from typing import cast
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
from matplotlib import colormaps as cm
from matplotlib.ticker import LinearLocator
import os
from mpl_toolkits.mplot3d import Axes3D
from typing import cast

ifplot = True
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
colors = ['b', 'r', 'g', 'y']

matplotlib.rc('axes', labelsize=25)
matplotlib.rc('xtick', labelsize=25)
matplotlib.rc('ytick', labelsize=25)
rc('text', usetex=True)


def initc(x, y):  # Initial condition

    u0 = np.ones((Mx, My))
    for i in range(Mx):
        for j in range(My):
            X, Y = x[i], y[j]
            u0[i, j] = np.sin(X+Y)+np.sin(X)+np.sin(Y)

    return u0


def wavenum(Mx, My):  # Wavenumber in Fourier space

    dk = np.pi/l
    kx = np.hstack((np.arange(0., Mx/2.+1.), np.arange(-Mx/2.+1., 0.))).T*dk
    ky = np.hstack((np.arange(0., My/2.+1.), np.arange(-My/2.+1., 0.))).T*dk

    return kx, ky


def weights(x, y):  # Spatial integration weights

    weights = np.zeros((Mx, My))
    nx = len(x)
    ny = len(y)
    dx = np.ones_like(x)
    dy = np.ones_like(y)

    for i in range(nx-1):
        dx[i] = x[i+1] - x[i]

    dx = np.delete(dx, [len(x)-1], None)

    for j in range(ny-1):
        dy[j] = y[j+1] - y[j]

    dy = np.delete(dy, [len(y)-1], None)

    for k in range(nx):
        for l in range(ny):
            if k == 0 and l == 0:
                weights[k, l] = dx[0]*dy[0]/4.
            elif k == 0 and l == ny-1:
                weights[k, l] = dx[0]*dy[-1]/4.
            elif k == nx-1 and l == 0:
                weights[k, l] = dx[-1]*dy[0]/4.
            elif k == nx-1 and l == ny-1:
                weights[k, l] = dx[-1]*dy[-1]/4.
            elif k == 0 and 0 < l < ny-1:
                weights[k, l] = dx[0]*(dy[l-1]+dy[l])/4.
            elif k == nx-1 and 0 < l < ny-1:
                weights[k, l] = dx[-1]*(dy[l-1]+dy[l])/4.
            elif 0 < k < nx-1 and l == 0:
                weights[k, l] = (dx[k-1]+dx[k])*dy[0]/4.
            elif 0 < k < nx-1 and l == ny-1:
                weights[k, l] = (dx[k-1]+dx[k])*dy[-1]/4.
            else:
                weights[k, l] = (dx[k-1]+dx[k])*(dy[l-1]+dy[l])/4.

    return weights


def kssol(u0):  # Solves Kuramoto-Sivashinsky equation in 2-D Fourier space

    kx, ky = wavenum(Mx, My)
    nkx = len(kx)
    nky = len(ky)

    k2 = np.ones((Mx, My))
    kX = np.ones((Mx, My))
    kY = np.ones((Mx, My))
    for i in range(nkx):
        for j in range(nky):
            KX, KY = kx[i], ky[j]
            k2[i, j] = -(KX**2)-((nu2/nu1)*(KY**2))+(nu1*((KX**4) +
                                                          (2.*(nu2/nu1)*(KX**2)*(KY**2))+((nu2/nu1)**2*(KY**4))))
            kX[i, j] = KX
            kY[i, j] = KY

    u0spec = np.fft.fft2(u0)  					# Initial condition in Fourier space

    # Nonlinear part x
    nlinspecx = np.zeros((Mx, My, nt+1), dtype='complex')
    # Nonlinear part y
    nlinspecy = np.zeros((Mx, My, nt+1), dtype='complex')
    NONLINEAR_PART = True
    if NONLINEAR_PART:
        nlinspecx[:, :, 0] = -0.5*np.fft.fft2(np.absolute(
            np.fft.ifft2(1j*kX*u0spec))*np.absolute(np.fft.ifft2(1j*kX*u0spec)))
        nlinspecy[:, :, 0] = -0.5*(nu2/nu1)*np.fft.fft2(np.absolute(
            np.fft.ifft2(1j*kX*u0spec))*np.absolute(np.fft.ifft2(1j*kX*u0spec)))
    A = np.ones((Mx, My))

    u = np.zeros((Mx, My, nt+1), dtype='complex')		# Variable in Fourier space
    u[:, :, 0] = u0spec
    ur = np.zeros((Mx, My, nt+1), dtype='complex')		# Variable in real space
    # Variable in real space (imaginary part) - created by Victor
    ui = np.zeros((Mx, My, nt+1), dtype='complex')
    ur[:, :, 0] = u0
    ui[:, :, 0] = u0.imag
    en = np.zeros((nt+1))					# Energy calculation
    wt = weights(x, y)
    ur2 = ur[:, :, 0]*ur[:, :, 0]
    # en = 0  # created by Victor, I don't care about energy
    en[0] = np.dot(wt.flatten(), ur2.flatten())
    nlin = np.zeros((nt+1))

    for i in range(nt):
        # if i % 10000 == 0:
        #     print(i)
        if i == 0:
            u[:, :, i+1] = (u[:, :, i] + (dt*(nlinspecx[:, :, i] +
                            nlinspecy[:, :, i] + (c*A*u[:, :, i]))))/(A + (dt*(k2+(c*A))))
            ur[:, :, i+1] = np.fft.ifft2(u[:, :, i+1]).real
            ur[:, :, i+1] = ur[:, :, i+1] - \
                ((1./(4.*(np.pi**2))) *
                 np.dot(wt.flatten(), ur[:, :, i+1].flatten())*A)
            ui[:, :, i+1] = np.fft.ifft2(u[:, :, i+1]).imag
            ur2 = ur[:, :, i+1]*ur[:, :, i+1]
            en[i+1] = np.dot(wt.flatten(), ur2.flatten())
        else:
            u[:, :, i] = np.fft.fft2(ur[:, :, i])
            if NONLINEAR_PART:
                nlinspecx[:, :, i] = -0.5*np.fft.fft2(np.absolute(np.fft.ifft2(
                    1j*kX*u[:, :, i]))*np.absolute(np.fft.ifft2(1j*kX*u[:, :, i])))
                nlinspecy[:, :, i] = -0.5*(nu2/nu1)*np.fft.fft2(np.absolute(
                    np.fft.ifft2(1j*kY*u[:, :, i]))*np.absolute(np.fft.ifft2(1j*kY*u[:, :, i])))
            u[:, :, i+1] = ((4.*u[:, :, i]) - u[:, :, i-1] + (4.*dt*(nlinspecx[:, :, i] + nlinspecy[:, :, i] + (c*A*u[:, :, i]))) - (
                2.*dt*(nlinspecx[:, :, i-1] + nlinspecy[:, :, i-1] + (c*A*u[:, :, i-1]))))/((3.*A) + (2.*dt*(k2+(c*np.ones_like(k2)))))
            ur[:, :, i+1] = np.fft.ifft2(u[:, :, i+1]).real
            ui[:, :, i+1] = np.fft.ifft2(u[:, :, i+1]).imag
            ur[:, :, i+1] = ur[:, :, i+1] - \
                ((1./(4.*(np.pi**2))) *
                 np.dot(wt.flatten(), ur[:, :, i+1].flatten())*A)
            ur2 = ur[:, :, i+1]*ur[:, :, i+1]
            en[i+1] = np.dot(wt.flatten(), ur2.flatten())
    return ur, ui, en


# Bifurcation parameters
nu1 = 0.8             							# nu1 = (pi/Lx)^2
nu2 = 0.2								# nu2 = (pi/Ly)^2

# Number of modes
Mx = 32                            					# Number of modes in x
My = 32                            					# Number of modes in y

# Coefficient to ensure the positive definiteness of linear matrix A
c = 100.

# Run time
Tf = 500.                          					# Final time
nt = 100000                         					# Number of time steps

Lx = np.pi/np.sqrt(nu1)                     				# Size of domain in x
Ly = np.pi/np.sqrt(nu2)	                     				# Size of domain in y

# Cell size
l = np.pi
dx = (2.*l)/(Mx)                    					# Grid spacing in x
dy = (2.*l)/(My) 							# Grid spacing in y

# Grid
x = np.arange(0., Mx)*dx       						# Grid points in x
y = np.arange(0., My)*dy						# Grid points in y
X, Y = np.meshgrid(x, y, indexing='ij')					# Meshgrid in x-y

# Step-size
dt = Tf/nt                        					# Size of the time step

t = np.linspace(0., Tf, nt+1)

u0 = initc(x, y)

ur, ui, en = kssol(u0)

# print ur completelly
# print(ur[:, :, -1])
# print("------------------")
# print(ui[:, :, -1])


/tmp/ipykernel_23051/1830890287.py:137: ComplexWarning: Casting complex values to real discards the imaginary part
  en[0] = np.dot(wt.flatten(), ur2.flatten())
/tmp/ipykernel_23051/1830890287.py:152: ComplexWarning: Casting complex values to real discards the imaginary part
  en[i+1] = np.dot(wt.flatten(), ur2.flatten())
/tmp/ipykernel_23051/1830890287.py:168: ComplexWarning: Casting complex values to real discards the imaginary part
  en[i+1] = np.dot(wt.flatten(), ur2.flatten())


In [8]:
# print entire np array: en (but without the 3 dots)
import sys
np.set_printoptions(threshold=sys.maxsize)
en

array([ 54.2682663 ,  54.31630322,  54.37734759,  54.4439823 ,
        54.51306783,  54.5830314 ,  54.65309408,  54.72287852,
        54.79221112,  54.8610219 ,  54.92929355,  54.99703555,
        55.06427081,  55.13102884,  55.1973422 ,  55.26324463,
        55.32876999,  55.3939518 ,  55.4588229 ,  55.52341525,
        55.58775991,  55.6518869 ,  55.71582527,  55.779603  ,
        55.84324706,  55.90678342,  55.97023703,  56.03363188,
        56.09699097,  56.16033641,  56.22368939,  56.28707021,
        56.35049835,  56.41399245,  56.47757039,  56.54124926,
        56.60504545,  56.66897463,  56.73305181,  56.79729136,
        56.86170702,  56.92631195,  56.99111875,  57.05613947,
        57.12138567,  57.18686838,  57.25259819,  57.31858523,
        57.38483922,  57.45136946,  57.51818485,  57.58529394,
        57.65270494,  57.72042569,  57.78846373,  57.8568263 ,
        57.92552036,  57.99455256,  58.06392933,  58.13365683,
        58.20374098,  58.27418749,  58.34500184,  58.41

In [ ]:
# grid of 16x16 in [0,2*pi]x[0,2*pi]
x = np.linspace(0, 2*np.pi, 16, endpoint=False)
y = np.linspace(0, 2*np.pi, 16, endpoint=False)
X, Y = np.meshgrid(x, y)


In [ ]:
np.sin(X) + np.sin(Y) + np.sin(X+Y)

array([[ 0.00000000e+00,  7.65366865e-01,  1.41421356e+00,
         1.84775907e+00,  2.00000000e+00,  1.84775907e+00,
         1.41421356e+00,  7.65366865e-01,  2.44929360e-16,
        -7.65366865e-01, -1.41421356e+00, -1.84775907e+00,
        -2.00000000e+00, -1.84775907e+00, -1.41421356e+00,
        -7.65366865e-01],
       [ 7.65366865e-01,  1.47247365e+00,  2.01366975e+00,
         2.30656296e+00,  2.30656296e+00,  2.01366975e+00,
         1.47247365e+00,  7.65366865e-01,  2.22044605e-16,
        -7.07106781e-01, -1.24830288e+00, -1.54119610e+00,
        -1.54119610e+00, -1.24830288e+00, -7.07106781e-01,
        -8.55552023e-16],
       [ 1.41421356e+00,  2.01366975e+00,  2.41421356e+00,
         2.55486585e+00,  2.41421356e+00,  2.01366975e+00,
         1.41421356e+00,  7.07106781e-01,  1.11022302e-16,
        -5.99456184e-01, -1.00000000e+00, -1.14065228e+00,
        -1.00000000e+00, -5.99456184e-01, -4.66973965e-16,
         7.07106781e-01],
       [ 1.84775907e+00,  2.30656296e

In [ ]:
np.cos(X) + np.cos(X+Y)

array([[ 2.00000000e+00,  1.84775907e+00,  1.41421356e+00,
         7.65366865e-01,  1.22464680e-16, -7.65366865e-01,
        -1.41421356e+00, -1.84775907e+00, -2.00000000e+00,
        -1.84775907e+00, -1.41421356e+00, -7.65366865e-01,
        -3.67394040e-16,  7.65366865e-01,  1.41421356e+00,
         1.84775907e+00],
       [ 1.92387953e+00,  1.63098631e+00,  1.08979021e+00,
         3.82683432e-01, -3.82683432e-01, -1.08979021e+00,
        -1.63098631e+00, -1.92387953e+00, -1.92387953e+00,
        -1.63098631e+00, -1.08979021e+00, -3.82683432e-01,
         3.82683432e-01,  1.08979021e+00,  1.63098631e+00,
         1.92387953e+00],
       [ 1.70710678e+00,  1.30656296e+00,  7.07106781e-01,
         1.11022302e-16, -7.07106781e-01, -1.30656296e+00,
        -1.70710678e+00, -1.84775907e+00, -1.70710678e+00,
        -1.30656296e+00, -7.07106781e-01, -1.16573418e-15,
         7.07106781e-01,  1.30656296e+00,  1.70710678e+00,
         1.84775907e+00],
       [ 1.38268343e+00,  9.23879533e

In [ ]:
np.cos(Y) + np.cos(X+Y)

array([[ 2.00000000e+00,  1.92387953e+00,  1.70710678e+00,
         1.38268343e+00,  1.00000000e+00,  6.17316568e-01,
         2.92893219e-01,  7.61204675e-02,  0.00000000e+00,
         7.61204675e-02,  2.92893219e-01,  6.17316568e-01,
         1.00000000e+00,  1.38268343e+00,  1.70710678e+00,
         1.92387953e+00],
       [ 1.84775907e+00,  1.63098631e+00,  1.30656296e+00,
         9.23879533e-01,  5.41196100e-01,  2.16772751e-01,
         0.00000000e+00, -7.61204675e-02, -1.11022302e-16,
         2.16772751e-01,  5.41196100e-01,  9.23879533e-01,
         1.30656296e+00,  1.63098631e+00,  1.84775907e+00,
         1.92387953e+00],
       [ 1.41421356e+00,  1.08979021e+00,  7.07106781e-01,
         3.24423349e-01,  1.11022302e-16, -2.16772751e-01,
        -2.92893219e-01, -2.16772751e-01, -1.11022302e-16,
         3.24423349e-01,  7.07106781e-01,  1.08979021e+00,
         1.41421356e+00,  1.63098631e+00,  1.70710678e+00,
         1.63098631e+00],
       [ 7.65366865e-01,  3.82683432e